Questo programma calcola la likelihood poissoniana associata al numero di ammassi di galassia ad un certo redshift e con una certa massa, al variare della cosmologia (in particolare facendo variare in modo crescente $\Omega_m$ tra 0.15 e 0.45.

Importo le funzioni utili allo svolgimento del lavoro:

In [1]:
import numpy as np
#import pylab as plt
import pyccl as ccl
import matplotlib.pyplot as plt
import emcee
from astropy.cosmology import Planck13
from astropy.cosmology import FlatLambdaCDM as FLCDM
%matplotlib inline

Definisco una funzione che mi permette di inserire solo i label al centro del bin, nei plot:

In [2]:
def bins_labels(bins, **kwargs):
    bin_w = 0.025
    plt.xticks(np.arange(min(bins)+bin_w, max(bins)+bin_w, 2*bin_w), bins, **kwargs)

Leggo i dati dalla tabella cat.txt (formata da due colonne), e li assegno a due array, M e z:

In [3]:
GCdata = np.loadtxt('cat.txt', usecols=(0,1))
#print(GCdata.shape)

M = GCdata[:,0]
z = GCdata[:,1]

#print(len(M), len(z))
#plt.scatter(M, 1+z)

Definisco un array con i valori di $\Omega_m$ che definiranno le diverse cosmologie. 
Definisco un binning in redshift e in massa, calcolo la likelihood poissoniana associata a ogni redshift per ogni cosmologia e poi plotto questi valori per fare un confronto.
La forma della likelihood è:

\begin{equation}
\log(L) = \sum_{i=M_{bin}} (n_i\log(\lambda_i)-\lambda_i)\ \ \forall z_{bin}, 
\end{equation}

data $P_{\lambda}(n)=\frac{\lambda^n}{n!}e^{-\lambda}$; dove $n_i$ è il numero di ammassi per bin di z e di massa, $\lambda_i$ è il valore teorico asssociato, calcolato grazie alla routine pyccl. Escludo il termine $-\log(n!)$

## Problema (forse):
Alla fine della funzione sommo la likelihood calcolata per ogni redshift perchè emcee voleva un solo valore per logl, è giusto? 

In [ ]:
def log_likelihood(theta, z, M):
    
    for r in theta:    
        cosmo = ccl.Cosmology(Omega_c=0.86*r, Omega_b=0.14*r, h=0.67, A_s=2.1e-9, n_s=0.96)
        m200c = ccl.halos.MassDef200c()
        cs = ccl.halos.MassFuncTinker08(cosmo, mass_def=m200c)

        cosmo_vol = FLCDM(H0 = 67.3, Om0 = r)

        z_bin = np.arange(0.05, 2.15, 0.1)
        #print(z_bin)
        M_bin = np.arange(14.025, 15.525, 0.05)
        #print(M_bin)

        bins = np.round(np.arange(14,15.6,0.05),3)

        i = 0
        k = 0
        log_l = []
        
        for red in z_bin:
            M_z = M[(z>=red-0.05) & (z<red+0.05)]
            plt.figure()
            n_obs, bins_hist, patches = plt.hist(M_z, bins, edgecolor = 'r', facecolor = 'red', alpha = 0.2)

            N = []
            log_p = []
            j = 0
            for mass in M_bin:
                ind_exp = np.arange(mass-0.025, mass+0.025,0.005)
                masses = 10**ind_exp
                nm = cs.get_mass_function(cosmo, masses, 1/(1+red))
                v_min = cosmo_vol.comoving_volume(red-0.05).value 
                v_max = cosmo_vol.comoving_volume(red+0.05).value 
                v_shell = (v_max-v_min)*0.36
                #print(v_max, v_min, v_shell)
                Nm = nm*v_shell
                k = np.trapz(Nm, x = ind_exp)
                N.append(k)
                log_p.append(n_obs[j]*np.log(k)-k)
                j += 1
            log_l.append(np.sum(log_p))  
            #bins_labels(bins, fontsize = 10)
            #plt.scatter(M_bin, N, marker = '.')
            #plt.savefig('mass_distr_%d.png' %i, bbox_inches = 'tight')
            plt.close()
            i += 1
        #print(log_l)
        
        #plt.scatter(z_bin, log_l, label='Om = %.2f'%r) 
        #plt.xlabel('z')
        #plt.ylabel('Log(L)')
        #plt.legend()
        #plt.savefig('likel_om_Tink.png', bbox_inches = 'tight')
        log_l = np.sum(log_l)
        
    return(log_l)

In [ ]:
def log_prior(theta):
    Om0 = theta
    if  Om0 > 0:
        return 0.0
    return -np.inf

In [ ]:
def log_posterior(theta, z, M):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    logl = log_likelihood(theta, z, M)
    logpos = logl+lp
    return logpos, lp, logl

In [ ]:
pos = [0.3] + 1e-4 * np.random.randn(16, 1)
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(z, M))
sampler.run_mcmc(pos, 2000, progress=True);

In ogni caso emcee sta andando, spero facendo le cose giuste; credo che ci vorrà un po' perchè è molto lento